# Unsupervised VAE analysis & further retraining the last layer

In [1]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import pandas as pd
import importlib
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
import optuna
import imaplib


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import random
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score


#importing fundtion to get encoder info and perfrom tasks 
from finetune.get_finetune_encoder import  get_finetune_input_data

from models.models_VAE import VAE
from pretrain.train_pretrain_VAE import PretrainVAE

%who



[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
2024-10-02 21:38:55.247002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 21:38:55.269579: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 21:38:55.276383: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 21:38:55.293724: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions

DataLoader	 LabelEncoder	 PretrainVAE	 TensorDataset	 VAE	 accuracy_score	 f1_score	 get_finetune_input_data	 imaplib	 
importlib	 label_encoder	 ml_code_path	 nn	 np	 optim	 optuna	 os	 pd	 
precision_score	 random	 recall_score	 roc_auc_score	 torch	 


## Evaluating unsupervised models

#### getting the input data

In [2]:
#input data
input_data_location='/home/leilapirhaji/PROCESSED_DATA_finetune_OG_split'

finetune_save_dir='/home/leilapirhaji/finetune_unsupervised_VAE'
pretrain_name='pretrain_VAE_L_400_425_e_400_p_25_s_8.1.1'
pretrain_trial_list=[143]
pretrain_trial_id=pretrain_trial_list[0]

#tasks to predict using encoder
task_list_cat=['Benefit BINARY', 'Nivo Benefit BINARY', 'MSKCC BINARY', 'IMDC BINARY', 'Benefit ORDINAL', 'MSKCC ORDINAL', 'IMDC ORDINAL', 'ORR', 'Benefit', 'IMDC', 'MSKCC', 'Prior_2' ]

#survival tasks
task_list_survival=[ 'OS', 'NIVO OS', 'EVER OS', 'PFS']


#get fine-tuning input data 
(X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)=get_finetune_input_data(input_data_location)

X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, 

((443, 2736), (443, 58), (149, 2736), (149, 58))

### loading the fine-tune VAE models w/wo transfer leanrning

In [3]:
from finetune.retrain_finetune_VAE import get_finetune_VAE_TL_noTL

#first getting A list of pretrained models and their associated fine-tune 

(finetune_VAE_TL_OG, finetune_VAE_noTL_OG)=get_finetune_VAE_TL_noTL(finetune_save_dir, pretrain_name, pretrain_trial_id)

finetune_VAE_TL_OG.latent_size, finetune_VAE_TL_OG.num_hidden_layers, finetune_VAE_noTL_OG.latent_size, finetune_VAE_noTL_OG.num_hidden_layers

/home/leilapirhaji/mz_embed_engine/ml/finetune/retrain_finetune_VAE.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  finetune_VAE_TL=torch.load(finetune_VAE_TL_file)
/ho

(425, 2, 425, 2)

## Visualizing the latent space

In [ ]:
import matplotlib.pyplot as plt
import finetune.eval_finetune_latent_unsupervised_main

importlib.reload(finetune.eval_finetune_latent_unsupervised_main)
from finetune.eval_finetune_latent_unsupervised_main import visualize_latent_space_multiple_tasks

models_path=f'{finetune_save_dir}/{pretrain_name}/trial_{pretrain_trial_id}'

result_png_file_TL= f'{models_path}/unsupervised_finetune_VAE_TL_latent_space.png'

visualize_latent_space_multiple_tasks(finetune_VAE_TL_OG, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test, result_png_file_TL)

In [ ]:
result_png_file_TL= f'{models_path}/unsupervised_finetune_VAE_noTL_latent_space.png'

visualize_latent_space_multiple_tasks(finetune_VAE_noTL_OG, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test, result_png_file_TL)

## Perdciting classification tasks from latnet space

In [70]:
#tasks to predict using encoder
task_list_cat=['Benefit BINARY', 'Nivo Benefit BINARY', 'MSKCC BINARY', 'IMDC BINARY', 'Benefit ORDINAL', 'MSKCC ORDINAL', 'IMDC ORDINAL', 'ORR', 'Benefit', 'IMDC', 'MSKCC', 'Prior_2' ]

task= 'MSKCC BINARY'


import finetune.eval_finetune_latent_unsupervised_main

importlib.reload(finetune.eval_finetune_latent_unsupervised_main)
from finetune.eval_finetune_latent_unsupervised_main import predict_task_from_latent_avg



best_val_accuracy, best_val_auc, test_accuracy, test_auc= predict_task_from_latent_avg (finetune_VAE_TL_OG, task, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test,y_data_test, batch_size=64, num_times=10)

best_val_accuracy, best_val_auc, test_accuracy, test_auc

Test Accuracy with best model: 0.8095
Test AUC with best model: 0.9014


(0.8292682926829268,
 0.8472222222222222,
 0.8095238095238095,
 0.9013998782714546)

In [71]:
best_val_accuracy, best_val_auc, test_accuracy, test_auc= predict_task_from_latent_avg (finetune_VAE_noTL_OG, task, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test,y_data_test, batch_size=64, num_times=10)

test_accuracy, test_auc

Test Accuracy with best model: 0.7857
Test AUC with best model: 0.8807


(0.7857142857142857, 0.8807060255629946)

## Predicting Survival from latnent avergae

### OS prediction

In [72]:
import finetune.eval_finetune_latent_unsupervised_main
importlib.reload(finetune.eval_finetune_latent_unsupervised_main)
from finetune.eval_finetune_latent_unsupervised_main import predict_survival_from_latent_avg


task= 'OS'
task_event='OS_Event'

best_val_c_index, best_test_c_index, best_params= predict_survival_from_latent_avg (finetune_VAE_TL_OG, task, task_event, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test,y_data_test, batch_size=64, num_times=10)

best_val_c_index, best_test_c_index, best_params

/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)


(0.6933306263323521, 0.6581597049147643, 0.01)

In [73]:
best_val_c_index, best_test_c_index, best_params= predict_survival_from_latent_avg (finetune_VAE_noTL_OG, task, task_event, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test,y_data_test, batch_size=64, num_times=10)

best_val_c_index, best_test_c_index, best_params

/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)


(0.7028728047913917, 0.6478915362376633, 0.01)

# Re-training the last layer of unsupervised VAE models

## grid search par opt

In [ ]:
!cd /home/leilapirhaji/mz_embed_engine/ml/

# Define common variables
INPUT_DATA_LOCATION="/home/leilapirhaji/PROCESSED_DATA_finetune_OG_split"
FINETUNE_SAVE_DIR="/home/leilapirhaji/finetune_unsupervised_VAE"

pretrain_model_name='pretrain_VAE_L_400_425_e_400_p_25_s_8.1.1'
pretrain_trial_ID='143'
TASK_EVENT='OS_Event'

# TASK="EVER OS"
# TASK_TYPE="cox"

TASK='MSKCC ORDINAL'
TASK_TYPE='classification'
num_classes='3'


# Run the finetune_VAE_local_main.py script
!python ./finetune/retrain_finetune_VAE_main.py \
    --input_data_location $INPUT_DATA_LOCATION \
    --finetune_save_dir $FINETUNE_SAVE_DIR \
    --pretrain_model_name $pretrain_model_name \
    --pretrain_trial_ID $pretrain_trial_ID \
    --task "$TASK" \
    --task_type $TASK_TYPE \
    --num_classes $num_classes \
    --task_event $TASK_EVENT \
    --optimization_type 'grid_search' \
    --add_post_latent_layers 'Fasle'\
    --post_latent_layer_size '64' \
    --num_layers_to_retrain '1' \
    --dropout_rate 0.4 \
    --learning_rate 5e-4 \
    --l1_reg 1e-6 \
    --weight_decay 1e-3 \
    --batch_size 32 \
    --patience 0 \
    --num_epochs 20 \
    --n_trials 3

In [19]:
#open the results of grid search
#reading the output
FINETUNE_SAVE_DIR="/home/leilapirhaji/finetune_unsupervised_VAE"
pretrain_model_name='pretrain_VAE_L_400_425_e_400_p_25_s_8.1.1'
pretrain_trial_ID='143'
TASK='EVER OS'
task=TASK.replace(' ','_')

result_path=f'{FINETUNE_SAVE_DIR}/{pretrain_model_name}/trial_{pretrain_trial_ID}/{task}'

#best paramteres
with open(f'{result_path}/{task}_best_params_grid_search.txt', 'r') as f:
    best_params = f.read()        

#best model results
best_model_results=pd.read_csv(f'{result_path}/{task}_best_model_results_grid_search.csv')
    
#All grid search results
all_grid_search_results= pd.read_csv(f'{result_path}/{task}_all_grid_search_results.csv')
#all_grid_search_results.sort_values('val_metric_TL', ascending=False, inplace=True)
# Convert a column (e.g., column 'B') to scientific notation
all_grid_search_results['learning_rate'] = all_grid_search_results['learning_rate'].apply(lambda x: '{:.2e}'.format(x))

all_grid_search_results

,add_post_latent_layers,post_latent_layer_size,num_layers_to_retrain,dropout,learning_rate,l1_reg_weight,l2_reg_weight,batch_size,patience,num_epochs,val_c_index_TL,val_c_index_noTL
0,False,64,1,0.4,1.00e-06,0.000001,0.001,32,0,20,0.724059,0.674472
1,False,64,1,0.4,3.00e-06,0.000001,0.001,32,0,20,0.753444,0.724059
2,False,64,1,0.4,5.00e-06,0.000001,0.001,32,0,20,0.760790,0.734160
3,False,64,1,0.4,7.00e-06,0.000001,0.001,32,0,20,0.765381,0.742883
4,False,64,1,0.4,9.00e-06,0.000001,0.001,32,0,20,0.768136,0.742424
5,False,64,1,0.4,1.00e-05,0.000001,0.001,32,0,20,0.768595,0.742883
6,False,64,1,0.4,3.00e-05,0.000001,0.001,32,0,20,0.773186,0.749311
7,False,64,1,0.4,5.00e-05,0.000001,0.001,32,0,20,0.770432,0.752066
8,False,64,1,0.4,7.00e-05,0.000001,0.001,32,0,20,0.769513,0.752525
9,False,64,1,0.4,9.00e-05,0.000001,0.001,32,0,20,0.767218,0.752525


In [20]:
best_model_results

,Unnamed: 0,Model,Dataset,C-index
0,0,Transfer Learning,Train,0.749240
1,1,Transfer Learning,Validation,0.773186
2,2,Transfer Learning,Test,0.658278
3,3,No Transfer Learning,Train,0.706099
4,4,No Transfer Learning,Validation,0.749311
5,5,No Transfer Learning,Test,0.657837


In [114]:
best_params

"{'add_post_latent_layers': False, 'post_latent_layer_size': 64, 'num_layers_to_retrain': 2, 'dropout': 0.4, 'learning_rate': 0.0001, 'l1_reg_weight': 1e-06, 'l2_reg_weight': 0.001, 'batch_size': 32, 'patience': 0, 'num_epochs': 20}"

### Plotting the latnet space - survival 

In [ ]:
import matplotlib.pyplot as plt
from finetune.best_finetune_model_test_eval_cox import best_model_latent_plot, best_finetune_model_test_eval_cox, best_model_latent_plot_combined

TASK_EVENT='OS_Event'
TASK='EVER OS'
task=TASK.replace(' ','_')

model_path=f'{finetune_save_dir}/{pretrain_name}/trial_{pretrain_trial_id}/{task}/TL_{task}_best_model_grid_search'
best_model_file=f'{model_path}/best_model.pth'
best_model=torch.load(best_model_file)

result_png_file_TL= f'{model_path}/{task}_best_model_TL_latent_space.png'


combined_result_metrics, latent_rep_train, latent_rep_val, latent_rep_test= best_finetune_model_test_eval_cox(best_model, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test, TASK, TASK_EVENT, seed=42)

fig, axes= best_model_latent_plot(latent_rep_train, latent_rep_val, latent_rep_test, y_data_train, y_data_val, y_data_test, TASK, TASK_EVENT)

fig.savefig(result_png_file_TL)
plt.show()

In [ ]:
result_png_file_TL= f'{model_path}/{task}_best_model_TL_latent_space_combined.png'
fig, axes= best_model_latent_plot_combined(latent_rep_train, latent_rep_val, latent_rep_test, y_data_train, y_data_val, y_data_test, TASK, TASK_EVENT)

fig.savefig(result_png_file_TL)

In [ ]:
model_path=f'{finetune_save_dir}/{pretrain_name}/trial_{pretrain_trial_id}/{task}/noTL_{task}_best_model_grid_search'
best_model_file=f'{model_path}/best_model.pth'
best_model_noTL=torch.load(best_model_file)

result_png_file_noTL= f'{model_path}/{task}_best_model_noTL_latent_space.png'


combined_result_metrics, latent_rep_train, latent_rep_val, latent_rep_test= best_finetune_model_test_eval_cox(best_model_noTL, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test, TASK, TASK_EVENT, seed=42)

fig, axes= best_model_latent_plot(latent_rep_train, latent_rep_val, latent_rep_test, y_data_train, y_data_val, y_data_test, TASK, TASK_EVENT)

fig.savefig(result_png_file_noTL)


In [ ]:
result_png_file_noTL= f'{model_path}/{task}_best_model_noTL_latent_space_combined.png'
fig, axes= best_model_latent_plot_combined(latent_rep_train, latent_rep_val, latent_rep_test, y_data_train, y_data_val, y_data_test, TASK, TASK_EVENT)

fig.savefig(result_png_file_noTL)

### Plotting the latnet space - classfication 

In [ ]:
import matplotlib.pyplot as plt
from finetune.best_finetune_model_test_eval import best_model_latent_plot, evaluate_model_main, best_model_latent_plot_combined

TASK='MSKCC BINARY'
num_classes='3'
task=TASK.replace(' ','_')

model_path=f'{finetune_save_dir}/{pretrain_name}/trial_{pretrain_trial_id}/{task}/TL_{task}_best_model_grid_search'
best_model_file=f'{model_path}/best_model.pth'
best_model_TL=torch.load(best_model_file)

result_png_file_TL= f'{model_path}/{task}_best_model_TL_latent_space.png'


combined_result_metrics, latent_rep_train, latent_rep_val, latent_rep_test= evaluate_model_main(best_model_TL, X_data_train, X_data_val, X_data_test, y_data_train, y_data_val, y_data_test, TASK, int(num_classes), seed=42)

fig, axes= best_model_latent_plot(latent_rep_train, latent_rep_val, latent_rep_test, y_data_train, y_data_val, y_data_test, TASK)

fig.savefig(result_png_file_TL)

In [ ]:
result_png_file_TL= f'{model_path}/{task}_best_model_TL_latent_space_combined.png'

fig, axes=best_model_latent_plot_combined(latent_rep_train, latent_rep_val, latent_rep_test, y_data_train, y_data_val, y_data_test, TASK)

fig.savefig(result_png_file_TL)

In [ ]:
model_path=f'{finetune_save_dir}/{pretrain_name}/trial_{pretrain_trial_id}/{task}/noTL_{task}_best_model_grid_search'
best_model_file=f'{model_path}/best_model.pth'
best_model_noTL=torch.load(best_model_file)

result_png_file_noTL= f'{model_path}/{task}_best_model_noTL_latent_space.png'

combined_result_metrics, latent_rep_train, latent_rep_val, latent_rep_test= evaluate_model_main(best_model_noTL, X_data_train, X_data_val, X_data_test, y_data_train, y_data_val, y_data_test, TASK, int(num_classes), seed=42)

fig, axes= best_model_latent_plot(latent_rep_train, latent_rep_val, latent_rep_test, y_data_train, y_data_val, y_data_test, TASK)

fig.savefig(result_png_file_noTL)


In [ ]:
result_png_file_noTL= f'{model_path}/{task}_best_model_noTL_latent_space_combined.png'

fig, axes=best_model_latent_plot_combined(latent_rep_train, latent_rep_val, latent_rep_test, y_data_train, y_data_val, y_data_test, TASK)

fig.savefig(result_png_file_noTL)